In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [85]:
import os

In [86]:
df = pd.read_excel(os.path.join("..","Data","SMC_DATA_clean.xlsx"))

In [87]:
platform_map = {
    "reddit": ["redit"],
    "snapchat": [],
    "twitter": [],
    "instagram": [],
    "facebook": [],
    "youtube": [],
    "tiktok": ["tik tok", "tik-tok"],
    "imessage": [],
    "discord": [],
    "facetime": ["facetiming", "face timing"],
    "pinterest": [],
}

for platform in platform_map.keys():
    platform_map[platform].append(platform)
    
# ignoring because out of scope: Slack, Spotify, Apple Music, Zoom, Netflix, Hulu, Disney+, HBO max, viki, Safari, email, Gmail
# excluding due to low prevalence ( <5% of participants, n=23):   
    # messenger     22.0
    # tinder         6.0
    # hinge          2.0
    # linkedin      21.0
    # bereal        18.0
    # vsco          15.0
    # whatsapp      15.0
    # groupme       19.0
    # weibo          1.0
    # wechat         5.0
    # yikyak         6.0
    # twitch         9.0

In [88]:
# make STATE_behaviors_and_goals lower case
df['STATE_behaviors_and_goals'] = df['STATE_behaviors_and_goals'].apply(lambda x: x.lower() if type(x) == str else x)

In [89]:
int(any(spelling in 'a b c d e' for spelling in ['x', 'y', 'z', 'a', 'b']))

1

In [90]:
for platform in platform_map.keys():
    platform_spellings = platform_map[platform]
    df[platform] = df['STATE_behaviors_and_goals'].apply(lambda x: int(any(spelling in x for spelling in platform_spellings)) if type(x) == str else x)
   

In [91]:
platforms = [p for p in platform_map.keys()]

In [92]:
df[df['SURVEY'] == 'INITIAL SURVEY'][platforms]

,reddit,snapchat,twitter,instagram,facebook,youtube,tiktok,imessage,discord,facetime,pinterest
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
5,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
10,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
15,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
20,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2175,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2179,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2188,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2196,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0


In [93]:
(df.groupby('SEMESTER')[platforms].sum()/92).loc['Fall 2022']

reddit       0.141304
snapchat     0.684783
twitter      0.195652
instagram    0.869565
facebook     0.173913
youtube      0.336957
tiktok       0.652174
imessage     0.130435
discord      0.141304
facetime     0.326087
pinterest    0.130435
Name: Fall 2022, dtype: float64

In [94]:
df[platforms].sum()/447

reddit       0.129754
snapchat     0.762864
twitter      0.324385
instagram    0.885906
facebook     0.228188
youtube      0.340045
tiktok       0.604027
imessage     0.129754
discord      0.170022
facetime     0.335570
pinterest    0.093960
dtype: float64

In [95]:
447 * 0.05

22.35

In [96]:
df[df['SURVEY'] == 'INITIAL SURVEY'][['SEMESTER','ParticipantID','UniqueID']].groupby('SEMESTER').count()['UniqueID']


SEMESTER
Fall 2020      76
Fall 2021      83
Fall 2022      92
Spring 2021    99
Spring 2022    97
Name: UniqueID, dtype: int64

In [97]:
76+83+92+99+97

447

In [106]:
#df.to_excel(os.path.join("..","Data","SMC_DATA_clean.xlsx"))

### Correlations

In [98]:
interest_cols = [col for col in df.columns if col.endswith('_total') or col.endswith('_score')]

In [99]:
interest_cols

['BSMAS_total',
 'PSS_total',
 'ADTS_ANX_total',
 'ADTS_P_total',
 'ADTS_N_total',
 'RSES_total',
 'SWLS_total',
 'PSOC_total',
 'LONE_total',
 'WEEKLY_self_assessment_score']

In [100]:
df_is = df[df['SURVEY'] == 'INITIAL SURVEY']

In [101]:
df_mean_scores = df[['ParticipantID'] + interest_cols].groupby('ParticipantID').mean().reset_index()

In [102]:
df_is[['ParticipantID'] + platforms].merge(df_mean_scores, on='ParticipantID').corr().loc[interest_cols][platforms].round(3)

,reddit,snapchat,twitter,instagram,facebook,youtube,tiktok,imessage,discord,facetime,pinterest
BSMAS_total,-0.072,0.091,0.123,0.079,0.061,-0.058,0.241,0.037,-0.096,0.046,0.033
PSS_total,0.008,-0.093,0.110,-0.103,-0.015,-0.039,0.064,-0.042,0.015,-0.025,0.048
ADTS_ANX_total,0.031,0.030,0.058,-0.067,-0.027,0.107,0.187,0.002,0.075,-0.060,0.044
ADTS_P_total,0.116,-0.034,0.026,-0.030,-0.020,0.112,-0.008,0.017,0.181,0.009,-0.029
ADTS_N_total,-0.017,0.048,-0.036,0.095,-0.031,-0.008,0.020,0.064,-0.224,0.164,-0.030
RSES_total,0.118,-0.218,0.137,-0.203,-0.039,0.008,-0.034,-0.021,0.052,-0.284,-0.060
SWLS_total,-0.090,0.265,-0.144,0.215,-0.002,-0.092,0.084,-0.008,-0.040,0.221,0.066
PSOC_total,-0.230,0.206,-0.055,0.248,0.023,-0.149,0.196,-0.059,-0.138,0.299,0.127
LONE_total,0.099,0.000,-0.073,-0.092,0.074,-0.006,0.065,0.027,-0.038,-0.115,-0.170
WEEKLY_self_assessment_score,-0.020,0.016,-0.062,-0.031,-0.017,0.072,-0.055,-0.024,0.001,-0.032,0.002


In [103]:
df_is[platforms].corr()

,reddit,snapchat,twitter,instagram,facebook,youtube,tiktok,imessage,discord,facetime,pinterest
reddit,1.000000,-0.176031,0.059525,-0.133653,-0.003726,0.088220,-0.082134,-0.089664,0.232851,-0.105224,-0.033078
snapchat,-0.176031,1.000000,0.015559,0.263152,0.077555,-0.110534,0.129348,0.090064,-0.209714,0.151164,0.017301
twitter,0.059525,0.015559,1.000000,0.023202,0.158425,0.027171,0.004066,0.016862,0.042575,0.094547,-0.010223
instagram,-0.133653,0.263152,0.023202,1.000000,0.077760,-0.217725,0.155470,0.033870,-0.174747,0.076211,0.043216
facebook,-0.003726,0.077555,0.158425,0.077760,1.000000,-0.097727,0.058743,-0.003726,-0.019048,0.042582,-0.028938
youtube,0.088220,-0.110534,0.027171,-0.217725,-0.097727,1.000000,-0.133372,0.060113,0.303680,-0.020069,0.011623
tiktok,-0.082134,0.129348,0.004066,0.155470,0.058743,-0.133372,1.000000,0.013156,-0.205867,0.091024,0.103958
imessage,-0.089664,0.090064,0.016862,0.033870,-0.003726,0.060113,0.013156,1.000000,-0.068432,0.204959,0.035375
discord,0.232851,-0.209714,0.042575,-0.174747,-0.019048,0.303680,-0.205867,-0.068432,1.000000,-0.145082,-0.043698
facetime,-0.105224,0.151164,0.094547,0.076211,0.042582,-0.020069,0.091024,0.204959,-0.145082,1.000000,-0.017764
